# EnumOutputParser

- 著者: [ranian963](https://github.com/ranian963)
- ピアレビュー: [JaeHo Kim](https://github.com/Jae-hoya)
- 校正: [Two-Jay](https://github.com/Two-Jay)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/07-EnumOutputParser.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/07-EnumOutputParser.ipynb)
## 概要
このチュートリアルでは、```EnumOutputParser```を使用してLLMの出力から**有効なEnum値を抽出**する方法を紹介します。

```EnumOutputParser```は、言語モデルの出力を**事前定義された列挙型(Enum)値の1つ**に解析するツールで、以下の機能を提供します:

- 列挙型解析: 文字列出力を事前定義された```Enum```値に変換します。
- 型安全性: 解析結果が常に定義された```Enum```値の1つであることを保証します。
- 柔軟性: スペースや改行を自動的に処理します。

### 目次

- [概要](#概要)
- [環境設定](#環境設定)
- [EnumOutputParserの紹介](#EnumOutputParserの紹介)
- [例: Colorsの列挙型パーサー](#例-Colorsの列挙型パーサー)

### 参考文献

- [LangChain: Output Parsers](https://python.langchain.com/docs/concepts/output_parsers/)
- [LangChain: EnumOutputParser](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.enum.EnumOutputParser.html#langchain.output_parsers.enum.EnumOutputParser)
- [Pythonの列挙型](https://docs.python.org/ja/3/library/enum.html)
- [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)
----

## 環境設定

環境をセットアップします。詳細については[環境設定](https://wikidocs.net/257836)を参照してください。

**[注意]**
- ```langchain-opentutorial```は、チュートリアル用の使いやすい環境設定、便利な関数、ユーティリティを提供するパッケージです。
- 詳細については[```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi)をご確認ください。

In [ ]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [ ]:
# 必要なライブラリをインストール
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai"
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
# 環境変数を設定
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "07-EnumOutputParser",
    }
)

あるいは、```.env```ファイルに```OPENAI_API_KEY```を設定して読み込むこともできます。

[注意] 前の手順で```OPENAI_API_KEY```を既に設定している場合、これは不要です。

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## EnumOutputParserの紹介

```EnumOutputParser```は、LLMの出力を定義された列挙型(Enum)に厳密に解析するツールです。
これにより、モデルの出力が常に列挙値の1つであることが保証されます。

**使用例**
- 可能性のセットから1つの有効な選択肢のみが必要な場合。
- 明確なEnum値を使用して、タイプミスやバリエーションを避けたい場合。

以下の例では、```Colors``` Enumを定義し、出力を解析してLLMに```red/green/blue```の1つを返させます。

## 例: Colorsの列挙型パーサー

以下のコードは、```Colors(Enum)```クラスを定義し、```EnumOutputParser```でラップしてから、プロンプトチェーンに統合する方法を示しています。
チェーンが実行されると、LLMの応答は**厳密に**```Colors```の値の1つに解析されます。

In [ ]:
# EnumOutputParserをインポート
from langchain.output_parsers.enum import EnumOutputParser

Pythonの組み込み```enum```モジュールの```Enum```クラスを使用して、```Colors```列挙型を定義します。

In [ ]:
from enum import Enum

class Colors(Enum):
    RED = "赤"
    GREEN = "緑"
    BLUE = "青"

次に、文字列を```Colors```列挙型に解析するための```EnumOutputParser```オブジェクトを作成します。

In [ ]:
# EnumOutputParserをインスタンス化
parser = EnumOutputParser(enum=Colors)

# パーサーが期待するフォーマット指示を表示できます
print(parser.get_format_instructions())

以下は、```PromptTemplate```と```ChatOpenAI```を使用して簡単なチェーンを構築する例です。
LLMが「空」というオブジェクトがどの色かについて応答すると、パーサーはその文字列を```Colors```値に変換します。

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# プロンプトテンプレート: パーサーのフォーマット指示を最後に追加
prompt = (
    PromptTemplate.from_template(
        """このオブジェクトは何色ですか?

オブジェクト: {object}

指示: {instructions}"""
    ).partial(instructions=parser.get_format_instructions())
)

# チェーン全体: (プロンプト) -> (LLM) -> (Enumパーサー)
chain = prompt | ChatOpenAI(temperature=0) | parser

それでは、チェーンを実行しましょう。

In [ ]:
response = chain.invoke({"object": "空"})
print("解析されたEnum:", response)
print("生のEnum値:", response.value)

## 追加の例

別の例として、サイズの列挙型を使ってみましょう。

In [ ]:
# サイズの列挙型を定義
class Size(Enum):
    SMALL = "小"
    MEDIUM = "中"
    LARGE = "大"

# パーサーを作成
size_parser = EnumOutputParser(enum=Size)

# プロンプトを作成
size_prompt = (
    PromptTemplate.from_template(
        """このアイテムのサイズを選択してください: {item}

{instructions}"""
    ).partial(instructions=size_parser.get_format_instructions())
)

# チェーンを作成
size_chain = size_prompt | ChatOpenAI(temperature=0) | size_parser

# 実行
result = size_chain.invoke({"item": "Tシャツ"})
print(f"選択されたサイズ: {result.value}")